# Import Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas  as pd
import seaborn as sns
from tensorflow.keras import layers as L
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix

# Data Preparation

In [ ]:
df_train=pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
df_test=pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')
y=df_test['label']

In [ ]:
y_train=df_train.pop('label')
X_train=df_train.copy()

In [ ]:
X_train=X_train.values.reshape(-1,28,28,1)

In [ ]:
y_test=df_test.pop('label')
X_test=df_test.copy()

In [ ]:
X_test=X_test.values.reshape(-1,28,28,1)

# Data Visualization

In [ ]:
f, ax = plt.subplots(3,3) 
f.set_size_inches(10, 10)
k = 0
for i in range(3):
    for j in range(3):
        ax[i,j].imshow(X_train[k,:,:].reshape(28, 28),cmap='gray')
        k += 1
    plt.tight_layout()    

In [ ]:
y_train=pd.DataFrame(y_train,columns=['label'])
y_train.head()

In [ ]:
X_train=X_train/255
X_test=X_test/255

In [ ]:
sns.countplot(y_train['label'])

# Build Model

In [ ]:
datagen=tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,
        horizontal_flip=True,vertical_flip=True)
datagen.fit(X_train)

In [ ]:
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

In [ ]:
model = tf.keras.Sequential()
model.add(L.Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (28,28,1)))
model.add(L.BatchNormalization())
model.add(L.MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(L.Dropout(0.2))
model.add(L.Flatten())
model.add(L.Dense(units = 512 , activation = 'relu'))
model.add(L.Dropout(0.3))
model.add(L.Dense(units = 24 , activation = 'softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

In [ ]:
lb= LabelBinarizer()
y_train=lb.fit_transform(y_train)
y_test=lb.fit_transform(y_test)

# Feed Model

In [ ]:
history=model.fit(X_train,y_train,batch_size=128,epochs=10,verbose=1,callbacks=[learning_rate_reduction],validation_split=0.1)

# Model Evaluation

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1.01])
plt.legend(loc='lower right')

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
predictions = model.predict_classes(X_test)
for i in range(len(predictions)):
    if(predictions[i] >= 9):
        predictions[i] += 1
predictions[:5]  

In [ ]:
cm = confusion_matrix(y,predictions)
cm = pd.DataFrame(cm , index = [i for i in range(25) if i != 9] , columns = [i for i in range(25) if i != 9])
plt.figure(figsize = (15,15))
sns.heatmap(cm,cmap= "Reds", linecolor = 'black' , linewidth = 1 , annot = True, fmt='')

# Sample Testing

In [ ]:
plt.figure(figsize=(14,7))
c=0
for i in range(9):
    plt.subplot(3,3,c+1)
    plt.imshow(X_test[i].reshape(28,28), cmap="gray",interpolation=None)
    plt.title("Predicted Class {}, Actual Class {}".format(predictions[i], y[i]))
    plt.tight_layout()
    c+=1

### Training Accuracy - 99.7%
### Testing Accuracy - 87%
## The End